In [1]:
import wandb
from types import SimpleNamespace
import yaml
#import openai
from openai import OpenAI, AsyncOpenAI
import os
import json
import pydantic
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.schema.runnable import RunnableMap


In [2]:
# to do
# add prompt template
# add output parser
# add batch / streaming

In [3]:
wandb_key = os.environ['WANDB_API_KEY']
openai_key = os.environ['OPENAI_CHATGPT_API_KEY']

project_name = 'untitled_llm_project_test'

#openai.api_key = openai_key

### Basic LangChain Example

In [4]:
# simple chain

prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "bears"})

c:\Users\james\github_repos\untitled_tbd_LLM_project\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
# user suppliec inputs

vectorstore = DocArrayInMemorySearch.from_texts(
    ["the sun is bright", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retriever.get_relevant_documents("what do bears like to eat")



In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

chain.invoke({"question": "what do bearks like to eat?"})

inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

inputs.invoke({"question": "where did harrison work?"})

In [ ]:
# basic openai connection

### Basic Open AI function example
* https://learn.deeplearning.ai/functions-tools-agents-langchain/lesson/3/langchain-expression-language-(lcel)

In [5]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

client = OpenAI(
  api_key=os.environ['OPENAI_CHATGPT_API_KEY'],  # this is also the default, it can be omitted
)

# response = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo-0613",
#     messages=messages,
#     functions=functions
# )

In [9]:
from openai import 

client = AsyncOpenAI(api_key=os.environ['OPENAI_CHATGPT_API_KEY'])

completion = await client.chat.completions.create(model='gpt-3.5-turbo',
                                       messages=messages)



AuthenticationError: Error code: 401 - {'error': {'message': 'You have insufficient permissions for this operation. Missing scopes: model.request', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
print(completion.choices[0].text)
print(dict(completion).get('usage'))
print(completion.model_dump_json(indent=2))